# Welcome!

In this notebook we introduce [Opvious](https://www.opvious.io), a _batteries-included optimization platform_, by walking through an end-to-end optimization example.

<div class="alert alert-block alert-info">
    &#9432; This example can be run directly from your browser when accessed via its <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=guides/welcome.ipynb">opvious.io/notebooks</a> URL. No Opvious account required.
</div>

Let's imagine that we are tasked with allocating a budget between various projects. We are also given an expected cost and value for each project. Our goal is to __pick projects__ which __maximize total value__ while keeping __total cost within the budget__. It turns out that our task can be formulated naturally as an [integer programming problem](https://en.wikipedia.org/wiki/Integer_programming) (it is actually an instance of the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)) which--unlike heuristics--will be guaranteed to give us an optimal allocation.

To get started, we install the [Opvious Python SDK](https://opvious.readthedocs.io).

In [1]:
%pip install opvious

## Formulating the problem

We now define our model using the SDK's [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html) which generates mathematical specifications from readable Python code. At a high level, each model contains four types of components:

* _Dimensions and parameters_, capturing the problem's inputs. Here, we have a single dimension (the list of available projects) and three parameters (the total budget, the cost of each project, and the value of each project).
* _Variables_, representing its outputs. We have a single output here, whether a project is selected or not (modeled as an indicator--0 or 1--variable).
* _Constraints_, enforcing invariants. Here we need to make sure that the sum of the selected projects' cost does not exceed the budget.
* _Objectives_, evaluating the quality of a solution. We have a single objective here: maximizing the sum of the selected projects' value.

Models are always _abstract_: they exist independently of data and can be reused with different input values. This separation allows them to be easily validated, exported, and version-controlled.

In [2]:
import opvious.modeling as om

class BudgetAllocation(om.Model):
    """A simple model allocating a budget between various projects"""
    
    # Inputs
    projects = om.Dimension()  # Set of available projects
    budget = om.Parameter.non_negative()  # Total budget value
    cost = om.Parameter.non_negative(projects)  # Cost per project
    value = om.Parameter.continuous(projects)  # Value of each project
    
    # Output
    selected = om.Variable.indicator(projects)  # Whether a project is selected (1) or not (0)
    
    @om.constraint
    def within_budget(self):
        """Ensure that the total (summed) cost of selected projects is less or equal to the budget"""
        yield om.total(self.selected(p) * self.cost(p) for p in self.projects) <= self.budget()
        
    @om.objective
    def maximize_value(self):
        """Maximize the total (summed) value of selected projects"""
        return om.total(self.selected(p) * self.value(p) for p in self.projects)
        
model = BudgetAllocation()

[`Model`](https://opvious.readthedocs.io/en/stable/api-reference.html#opvious.modeling.Model) instances expose various built-in methods. The most important one is `specification`, which automatically validates and exports its mathematical representation. This specification is integrated with IPython’s rich display capabilities, making it easy to review:

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">BudgetAllocation</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{projects}&: P \\
  \S^p_\mathrm{budget}&: b \in \mathbb{R}_+ \\
  \S^p_\mathrm{cost}&: c \in \mathbb{R}_+^{P} \\
  \S^p_\mathrm{value}&: v \in \mathbb{R}^{P} \\
  \S^v_\mathrm{selected}&: \sigma \in \{0, 1\}^{P} \\
  \S^c_\mathrm{withinBudget}&: \sum_{p \in P} \sigma_{p} c_{p} \leq b \\
  \S^o_\mathrm{maximizeValue}&: \max \sum_{p \in P} \sigma_{p} v_{p} \\
\end{align*}
$$
</div>
</details>
</div>

## Finding a solution

Once our model is instantiated, we are ready to add data and solve it!

Combining a model with data is straightforward: we just provide a value for each parameter. In this simple example we provide them as a number (budget) and dictionaries (project cost and value) but many other formats are accepted, including `pandas` series.

In [4]:
import opvious

problem = opvious.Problem(  # Sample problem instance with 3 projects
    model.specification(),
    parameters={
        "budget": 100,
        "cost": {"p1": 50, "p2": 45, "p3": 60},
        "value": {"p1": 5, "p2": 6, "p3": 10},
    },
)

Solves are handled remotely via [`Client`](https://opvious.readthedocs.io/en/stable/api-reference.html#opvious.Client) instances. Since our sample problem is small it can be solved without authentication, otherwise we would need to [configure our client accordingly](https://opvious.readthedocs.io/en/stable/overview.html#creating-a-client).

In [5]:
client = opvious.Client.default()

async def solve_problem(problem):
    """Returns an optimal set of projects for the provided budget allocation problem"""
    solution = await client.solve(problem)
    selected = solution.outputs.variable("selected")  # `pandas` dataframe containing optimal variable values
    return set(selected.index)  # Names of the selected projects

# Let's try it!
await solve_problem(problem)

{'p1', 'p2'}

The solution is what we expect: selecting `p1` and `p2` yields a total value of 11 (greater than `p3`'s 10), with a total cost of 95 (less than the budget of 100).

## Digging deeper

`solve` is just one of the many useful methods available on client instances. For example `format_problem` will return the problem’s fully annotated representation in [LP format](https://web.mit.edu/lpsolve/doc/CPLEX-format.htm). This can be used for quick sanity checks or as input to other optimization tools.

In [6]:
print(await client.format_problem(problem))

maximize
  +5 selected$1 \ [projects=p1]
  +6 selected$2 \ [projects=p2]
  +10 selected$3 \ [projects=p3]
subject to
 withinBudget$1:
  +50 selected$1 \ [projects=p1]
  +45 selected$2 \ [projects=p2]
  +60 selected$3 \ [projects=p3]
  <= +100
general
  selected$1 \ [projects=p1]
  selected$2 \ [projects=p2]
  selected$3 \ [projects=p3]
end



The client also exposes summary statistics about each solved problem, making it easy to spot potential issues (weight imbalances, performance bottlenecks, ...).

In [7]:
summary = await client.summarize_problem(problem)
summary.constraints # Summary statistics about the problem's constraints (count, sparsity, weight distribution, ...)

,row_count,row_sprs,column_count,column_sprs,weight_count,weight_min,weight_max,weight_mean,weight_stddev,weight_sprs,reify_ms
label,,,,,,,,,,,
withinBudget,1,0.0,3,0.0,3,45,60,51.666667,7.637626,0.0,1


## Next steps

+ Browse our other interactive [guides and examples](https://www.opvious.io/notebooks/retro)
+ Create a (free) account via the [Optimization Hub](https://hub.cloud.opvious.io) to solve larger problems
+ Try the platform out locally with a [self-hosted API server](https://hub.docker.com/r/opvious/api-server)